In [10]:
!pip install matplotlib 
!pip install opencv-python 


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 8.5 MB/s eta 0:00:05
   -- ------------------------------------- 2.4/39.5 MB 6.4 MB/s eta 0:00:06
   --- ------------------------------------ 3.7/39.5 MB 6.2 MB/s eta 0:00:06
   ---- ----------------------------------- 4.7/39.5 MB 5.7 MB/s eta 0:00:07
   ----- ---------------------------------- 5.5/39.5 MB 5.3 MB/s eta 0:00:07
   ------ --------------------------------- 6.6/39.5 MB 5.1 MB/s eta 0:00:07
   ------- -------------------------------- 7.3/39.5 MB 4.9 MB/s eta 0:00:07
   ------- -------------------------------- 7.9/39.5 MB 4.7 MB/s eta 0:00:07
   --------- ------------------------------ 8.9/39.5 MB 4.6 MB/s eta 0:00:07
   --------- ------------------------------ 9.4/39.5 MB 4.4 MB/s eta 0:00:07
   ---------- ----------------------------- 10.0/39.5 MB 4.2 MB/s eta 0:00:07
   ---------- ----------------------------- 10.5/39.5 MB 4.1 MB/s eta 0:00:08
   -


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
from skimage import morphology
from skimage.filters import threshold_otsu

In [ ]:
def load_reflectance_image(image_path):
    with rasterio.open(image_path) as src:
        img = src.read().astype(np.float32)
        meta = src.meta
    return img, meta

In [ ]:
def detect_clouds(image):
    rgb = np.transpose(image, (1, 2, 0))  # (H, W, 3)
    brightness = np.mean(rgb, axis=2)
    thresh = threshold_otsu(brightness)
    cloud_mask = (brightness > thresh).astype(np.uint8)
    cloud_mask = morphology.remove_small_objects(cloud_mask.astype(bool), 100).astype(np.uint8)
    cloud_mask = morphology.binary_closing(cloud_mask, morphology.disk(3)).astype(np.uint8)
    return cloud_mask


In [ ]:
def detect_shadows(image):
    rgb = np.transpose(image, (1, 2, 0))
    brightness = np.mean(rgb, axis=2)
    shadow_thresh = threshold_otsu(brightness) * 0.7  # lower than Otsu
    shadow_mask = (brightness < shadow_thresh).astype(np.uint8)
    shadow_mask = morphology.remove_small_objects(shadow_mask.astype(bool), 100).astype(np.uint8)
    shadow_mask = morphology.binary_opening(shadow_mask, morphology.disk(2)).astype(np.uint8)
    return shadow_mask

In [ ]:
def generate_and_save_mask(image_path, output_path):
    image, meta = load_reflectance_image(image_path)
    cloud = detect_clouds(image)
    shadow = detect_shadows(image)

    # Final mask: 0 = background, 1 = cloud, 2 = shadow
    final_mask = np.zeros(cloud.shape, dtype=np.uint8)
    final_mask[cloud == 1] = 1
    final_mask[shadow == 1] = 2

    meta.update(dtype='uint8', count=1)
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(final_mask, 1)

    print(f"✅ Mask saved to {output_path}")

In [ ]:
input_image = "data/processed/images/R2F01JAN2025_reflectance.tif"
output_mask = "data/processed/masks/R2F01JAN2025_mask.tif"

In [ ]:
with rasterio.open(output_mask) as msk:
    plt.imshow(msk.read(1), cmap='gray')
    plt.title("Generated Mask")
    plt.colorbar()
    plt.show()
